In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager(tbout='/is/ei/fleeb/workspace/trained_nets/managed')
M.clear_links()
M.start_tb()

/is/ei/fleeb/workspace/trained_nets/managed is available to view runs on tensorboard


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 173 runs
Tensorboard started: http://localhost:6006/


In [3]:
#M.show()
M.clear_filters().select(model=None, dataset='atari').filter(lambda r: not 'test' in r.name)
# M.filter(lambda r: '200108' in r.name or '200109' in r.name or '200110' in r.name)
# M.filter(lambda r: '191218' in r.name)

# M.filter_idx(7, 5, 27, 23, 34, 16, 15, 13, 30, 28, 10, 12, 18, 21, 0, 1, 72, 73, 74, 67, 66, 57)


M.show()

  0 - atari-fdwae_0144-5988348-03_200116-145417
  1 - atari-fdwae_0145-5988349-04_200116-145444
  2 - atari-fdwae_0160-5988664-03_200116-190055
  3 - atari-fdwae_0161-5988667-04_200116-190600
  4 - atari-fvae_0144-5988348-02_200116-145418
  5 - atari-fvae_0145-5988349-03_200116-145443
  6 - atari-fvae_0160-5988664-02_200116-190055
  7 - atari-fvae_0161-5988667-03_200116-190600
  8 - atari-vae_0144-5988348-01_200116-145450
  9 - atari-vae_0145-5988349-02_200116-145444
 10 - atari-vae_0160-5988664-01_200116-190056
 11 - atari-vae_0161-5988667-02_200116-190600
 12 - atari-wae_0144-5988348-00_200116-145418
 13 - atari-wae_0145-5988349-00_200116-145444
 14 - atari-wae_0145-5988349-01_200116-145523
 15 - atari-wae_0160-5988664-00_200116-190055
 16 - atari-wae_0161-5988667-00_200116-190600
 17 - atari-wae_0161-5988667-01_200116-190600
 18 - atari-wgan_0145-5988349-05_200116-145443
 19 - atari-wgan_0161-5988667-05_200116-190600


In [4]:
ignore = None
ignore = ['disc_steps']
M.show_unique(ignore)

0) atari-fdwae_0144-5988348-03_200116-145417
	model.nonlin - elu (prelu)
	model.scheduler_type - plateau (step)
	model.prob - 0.5 (1)
	model.down_type - max (_)
	model.up_type - bilinear (_)
	game - asterix (_)

1) atari-fdwae_0145-5988349-04_200116-145444
	model.nonlin - elu (prelu)
	model.scheduler_type - plateau (step)
	model.prob - 0.5 (1)
	model.down_type - max (_)
	model.up_type - bilinear (_)
	game - pacman (_)

2) atari-fdwae_0160-5988664-03_200116-190055
	training.epochs - 60 (100)
	dataset.batch_size - 64 (32)
	model.prob - 0.5 (1)
	game - asterix (_)
	scheduler_patience - 4 (_)

3) atari-fdwae_0161-5988667-04_200116-190600
	training.epochs - 60 (100)
	dataset.batch_size - 64 (32)
	model.prob - 0.5 (1)
	game - pacman (_)
	scheduler_patience - 4 (_)

4) atari-fvae_0144-5988348-02_200116-145418
	model.nonlin - elu (prelu)
	model.scheduler_type - plateau (step)
	model.down_type - max (_)
	model.up_type - bilinear (_)
	game - asterix (_)

5) atari-fvae_0145-5988349-03_200116-1454

In [8]:
# M.link('{idx}__{model}__{unique}')
M.link('{job}__{model}__{unique}')

Unable to get first event timestamp for run 0102-5966133-06__dwae__generator.splits=4__generator.adain_latent_dim=16__generator.full_latent_dim=16__prob=0p5: No event timestamp could be found


In [6]:
# M.link('{name}__{unique}')

In [6]:
# os.listdir(M.tbout)

In [9]:
M.clear_links()

In [8]:
M.tbout

'/is/ei/fleeb/workspace/trained_nets/managed'

In [9]:
M.active[0].keys()

odict_keys(['name', 'path', 'ckpt_path', 'config', 'base', 'diffs'])

In [10]:
M.active[0].config.info.job

{{'ID':'5966131', 'num':'100', 'ps':'7', '_ipython_canary_method_should_not_exist_':{{}}}}

In [15]:
M.active[0].config.name.split('_')[-1]

'0100-5966131-07'

In [7]:
root = os.environ['FOUNDATION_SAVE_DIR']
root

'/is/ei/fleeb/workspace/chome/trained_nets'

In [8]:
for r in M.active:
    n = r.name
    num = len(os.listdir(os.path.join(root, n)))
    if num > 100:
        print('{} : {}'.format(n, num))

3dshapes-dvae_0100-5966131-07_200109-133026 : 113
3dshapes-dvae_0101-5966132-01_200109-133045 : 114
3dshapes-dvae_0101-5966132-02_200109-133053 : 113
3dshapes-dvae_0102-5966133-07_200109-133118 : 113
3dshapes-dvae_0102-5966133-08_200109-133107 : 113
3dshapes-dvae_0122-5969012-01_200110-140610 : 113
3dshapes-dvae_0122-5969012-02_200110-140616 : 113
3dshapes-dwae_0094-5966044-04_200108-175514 : 113
3dshapes-dwae_0095-5966045-04_200108-175850 : 113
3dshapes-dwae_0096-5966047-04_200108-175913 : 113
3dshapes-dwae_0097-5966051-04_200108-181233 : 113
3dshapes-dwae_0100-5966131-05_200109-133027 : 113
3dshapes-dwae_0100-5966131-06_200109-133124 : 113
3dshapes-dwae_0101-5966132-00_200109-133044 : 113
3dshapes-dwae_0102-5966133-05_200109-133202 : 113
3dshapes-dwae_0102-5966133-06_200109-133106 : 113
3dshapes-dwae_0122-5969012-00_200110-140613 : 113
3dshapes-fdwae_0094-5966044-05_200108-175529 : 113
3dshapes-fdwae_0095-5966045-05_200108-175832 : 113
3dshapes-fdwae_0096-5966047-05_200108-175918 : 1

In [18]:
import json

In [19]:
json.loads('[10,2,3]')

[10, 2, 3]

In [18]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

'/is/ei/fleeb/workspace/chome/trained_nets'

In [19]:
for name in sorted(os.listdir(saveroot), key=lambda x: x.split('_')[1]):
    try:
        if 'checkpoint_100.pth.tar' not in os.listdir(os.path.join(saveroot, name)):
            print(name)
    except (FileNotFoundError, NotADirectoryError):
#         print('ERROR: {}'.format(name))
        pass

3dshapes-vae_0090-5965922-00_200108-160038
3dshapes-fwae_0090-5965922-01_200108-160043
3dshapes-dwae_0090-5965922-04_200108-160125
3dshapes-fdwae_0090-5965922-05_200108-160042
3dshapes-wgan_0090-5965922-06_200108-160031
3dshapes-vae_0092-5966021-00_200108-171843
3dshapes-fwae_0092-5966021-01_200108-171821
3dshapes-fvae_0092-5966021-02_200108-171821
3dshapes-wpp_0092-5966021-03_200108-171825
3dshapes-dwae_0092-5966021-04_200108-171824
3dshapes-fdwae_0092-5966021-05_200108-171822
3dshapes-wgan_0093-5966029-00_200108-172349
3dshapes-wpp_0098-5966052-00_200108-181326
3dshapes-wae_0099-5966123-00_200109-125342
3dshapes-dvae_0100-5966131-08_200109-133040
3dshapes-fvae_0106-5966166-00_200109-154555
redball-wpp_0132-5971295-04_200113-165212
redball-dwae_0132-5971295-05_200113-165215
redball-fdwae_0132-5971295-07_200113-165209
redball-fdwae_0132-5971295-08_200113-165209
redball-wae_0133-5971579-00_200114-120657
redball-vae_0133-5971579-01_200114-120657
redball-fwae_0133-5971579-02_200114-120657

In [ ]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

In [ ]:
data['parents']